In [5]:
import deeplake as dl

BATCH_NAME = 'yt1b-val'
# INPUT_DATASET_PATH = f'/mnt/teton/vpt/data/yt-1b_deeplake/feb_25_text_encode_results_{BATCH_NAME}'
RESULTS_DATASET_PATH = f'/mnt/teton/vpt/data/yt-1b_deeplake/feb_25_CLIP_encode_results_{BATCH_NAME}'


In [6]:
ds = dl.load(RESULTS_DATASET_PATH, read_only=False)
ds.summary()

/mnt/teton/vpt/data/yt-1b_deeplake/feb_25_CLIP_encode_results_yt1b-val loaded successfully.

Dataset(path='/mnt/teton/vpt/data/yt-1b_deeplake/feb_25_CLIP_encode_results_yt1b-val', tensors=['caption', 'caption_embedding', 'clip_last_hidden_states', 'clip_pooled_embedding', 'done_text_encode', 'frames', 'segment_metadata', 'timestamp', 'video_filename', 'video_filepath'])

         tensor            htype            shape             dtype  compression
         -------          -------          -------           -------  ------- 
         caption           text           (77058, 1)           str     None   
    caption_embedding     generic    (77058, 16:73, 1024)    float16   None   
 clip_last_hidden_states  generic     (77058, 577, 1024)     float32    lz4   
  clip_pooled_embedding   generic       (77058, 1024)        float32    lz4   
    done_text_encode      generic         (77058, 1)          bool     None   
         frames            image   (77058, 360, 176:640, 3)   uint8    

In [ ]:
total_errors = 0
for i, v in enumerate(ds):
  # print(v)
  try:
    ddd = v.clip_pooled_embedding.numpy()
    # ddd = v.clip_last_hidden_states.numpy()
  except Exception as e:
    # reset all values in this index.
    ds[i] = populate_ds_with_zeros(v)
    total_errors += 1
    print("INDEX: ", i, "error:", e)

print(total_errors) # 41889 for pooled
# starting index: 25_861
# end index 51613
print(ds.max_len)
print(ds.min_len)

## Potential good solution

In [7]:
import numpy as np
import psutil


# PROBLEM. When

@dl.compute
def populate_ds_with_zeros(sample_in, sample_out):
  '''
  This is to fix datasets that were somehow corrupted when the program crashed.
  I get index errors, so I just reset all tensors that are corrupted.
  
  
  Pre-populate the dataset with zeros of proper shape. This makes it 100x faster to update later via indexing. 
  '''
  try:
    # access all data to expose `IndexError: index 3 is out of bounds for axis 0 with size 3`
    d = sample_in.clip_pooled_embedding.numpy()
    d = sample_in.clip_last_hidden_states.numpy()
    d = sample_in.frames.numpy()
    d = sample_in.timestamp.numpy()
  except Exception as e:
    sample_out.clip_pooled_embedding.append(np.zeros(1024, dtype=np.float32))
    sample_out.clip_last_hidden_states.append(np.zeros((577, 1024), dtype=np.float32))
    sample_out.frames = np.zeros((360, 640, 3), dtype=np.uint8)
    sample_out.timestamp.append(float(0))
  return sample_out

populate_ds_with_zeros().eval(ds, scheduler="ray", num_workers=psutil.cpu_count(), skip_ok=True)
ds.summary()

2023-03-06 16:30:33,077	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
Evaluating populate_ds_with_zeros: 100%|██████████| 77058/77058 [10:49<00:00


Dataset(path='/mnt/teton/vpt/data/yt-1b_deeplake/feb_25_CLIP_encode_results_yt1b-val', tensors=['caption', 'caption_embedding', 'clip_last_hidden_states', 'clip_pooled_embedding', 'done_text_encode', 'frames', 'segment_metadata', 'timestamp', 'video_filename', 'video_filepath'])

         tensor            htype            shape             dtype  compression
         -------          -------          -------           -------  ------- 
         caption           text           (77058, 1)           str     None   
    caption_embedding     generic    (77058, 16:73, 1024)    float16   None   
 clip_last_hidden_states  generic      (277, 577, 1024)      float32    lz4   
  clip_pooled_embedding   generic        (277, 1024)         float32    lz4   
    done_text_encode      generic         (77058, 1)          bool     None   
         frames            image   (77058, 360, 176:640, 3)   uint8    jpeg   
    segment_metadata       text           (77058, 1)           str     None   
      

## Bfloat16

In [ ]:
import numpy as np
import tensorflow as tf

bfloat16 = tf.bfloat16.as_numpy_dtype

np.array([1.0, 2.0, 3.0], dtype=bfloat16)
# array([bfloat16(1), bfloat16(2), bfloat16(3)], dtype=bfloat16)